In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
train = pd.read_csv("../input/facebook-recruiting-iv-human-or-bot/train.csv")
test = pd.read_csv("../input/facebook-recruiting-iv-human-or-bot/test.csv")
bids = pd.read_csv("../input/facebook-recruiting-iv-human-or-bot/bids.csv")
train.drop(1122,inplace=True)
print(train.shape, test.shape, bids.shape)
train.head()

alldata = pd.concat([train, test], sort=False)
print(alldata.shape)
alldata.head()

alldata["payment_account"] = alldata.payment_account.apply(lambda x: str(x)[:5])
alldata["address"] = alldata.address.apply(lambda x: str(x)[:5])
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
alldata["payment_account"] = le.fit_transform(alldata["payment_account"]).astype(np.int16)
alldata["address"] = le.fit_transform(alldata["address"]).astype(np.int16)
alldata["outcome"] = alldata["outcome"].astype(np.float16)
import gc; gc.collect()

bids.drop(["bid_id"], 1, inplace=True)

work = bids.sort_values("bidder_id")
timework = work[["bidder_id", "time"]]
auc_count_nunique = work.groupby("bidder_id")["auction"].agg(["count", "nunique"]).reset_index()
alldata = alldata.merge(auc_count_nunique, how = "left", on = "bidder_id")
alldata["nunique_div_count"] = alldata["nunique"]/alldata["count"]

merchan = pd.DataFrame(work.groupby("bidder_id")["merchandise"].apply(lambda x: x.mode())).reset_index()
alldata = alldata.merge(merchan, how = "left", on="bidder_id")
alldata.drop("level_1",1,inplace=True)
alldata["merchandise"] = le.fit_transform(alldata["merchandise"].astype("str")).astype(np.int16)

dev_count_nunique = work.groupby("bidder_id")["device"].agg(["count", "nunique"]).reset_index()
alldata = alldata.merge(dev_count_nunique, how = "left", on = "bidder_id")
alldata["dev_nunique_div_count"] = alldata["nunique_y"]/alldata["count_y"]
alldata.drop("count_y",1,inplace=True)

alldata = alldata.merge(pd.DataFrame(work.groupby("bidder_id")["country"].nunique()).reset_index(), how="left", on="bidder_id")

ip_url_nunique = work.groupby("bidder_id")["ip", "url"].nunique().reset_index()
alldata = alldata.merge(ip_url_nunique, how = "left", on = "bidder_id")
alldata["ip_nuni_div_count"] = alldata["ip"]/alldata["count_x"]
alldata["url_nuni_div_count"] = alldata["url"]/alldata["count_x"]
alldata = alldata.merge(pd.DataFrame(timework.groupby("bidder_id")["time"].apply(lambda x:(x.max() - x.min()))).reset_index(), how = "left", on = "bidder_id")
alldata["time_div_count"] = alldata["time"] / alldata["count_x"]


    
k = pd.read_csv("../input/i-m-not-robot/timediff.csv")
alldata = alldata.merge(k, how = "left", on = "bidder_id")

train = alldata.iloc[:len(train), :]
test = alldata.iloc[len(train):, :]

y = train["outcome"]
train.drop(["outcome", "bidder_id"],1,inplace=True)
test.drop(["outcome", "bidder_id"],1,inplace=True)
train.fillna(-1, inplace=True)
test.fillna(-1, inplace=True)
train.head()

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 300, random_state = 42, n_jobs=-1, max_depth = 9)
rf.fit(train, y)
preds = rf.predict_proba(test)
sub = pd.read_csv("../input/facebook-recruiting-iv-human-or-bot/sampleSubmission.csv")
sub.prediction = preds[:,1]
sub.to_csv("ha.csv", index=False)

(2012, 4) (4700, 3) (7656334, 9)
(6712, 4)


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
